分枝定界法  
匈牙利法

0-1规划包括：指派问题、TSP问题、集合覆盖问题

### 匈牙利算法

指派问题（0-1整数规划）

In [1]:

import numpy as np
from scipy.optimize import linear_sum_assignment

cost =np.array([[4,1,3],[2,0,5],[3,2,2]])
row_ind,col_ind=linear_sum_assignment(cost)
print(row_ind)#开销矩阵对应的行索引
print(col_ind)#对应行索引的最优指派的列索引
print(cost[row_ind,col_ind])#提取每个行索引的最优指派列索引所在的元素，形成数组
print(cost[row_ind,col_ind].sum())#数组求和

[0 1 2]
[1 0 2]
[1 2 2]
5


### 蒙特卡洛法

In [9]:
# import numpy as np


# def check(x):
# 	if x.sum() > 400:
# 		return False
# 	if x[0]+2*x[1]+2*x[2]+x[3]+6*x[4] > 800:
# 		return False
# 	if 2*x[0]+x[1]+6*x[2]>200:
# 		return False
# 	if x[2]+x[3]+5*x[3]>200:
# 		return False

# 	return True


# def get_radom():
# 	x = np.random.randint(100, size=5)
# 	while not check(x):
# 		x = get_radom()
# 	return x


# lim = 10**6
# ans = -1

# for i in range(lim):
# 	num = get_radom()
# 	ans = max(ans, num.all())
# 	if i % 10000 == 0:
# 		print(i)

# print('ans=' + ans)

### 混合整数规划

分枝定界法

In [11]:
from scipy.optimize import linprog
import numpy as np
from math import floor, ceil
import copy
 
 
class Node(object):
    def __init__(self, x_bounds=[], freeze_var_list=[], index=0, upper_or_lower=0):
        self._x_bounds = x_bounds
        self._freeze_var_list = freeze_var_list
        self._index = index
        self._upper_or_lower = upper_or_lower
 
        print("创建节点: {}".format(index))
        print('')
 
    def freeze_var(self, index, val):
        self._x_bounds[index][0] = val
        self._x_bounds[index][1] = val
        self._freeze_var_list.append(index)
 
    def set_lp_res(self, res):
        self._res = res
        s = ""
        for l in range(len(self._res['x'])):
            if l in self._freeze_var_list:
                s += "[" + str(self._res['x'][l]) + "]"
            else:
                s += " " + str(self._res['x'][l])
        print("x: ", s)
 
    def check_integer_val_solved(self, m):
        return True if m == len(self._freeze_var_list) else False
 
 
class BbAlgorithm(object):
    def __init__(self, c, a_ub, b_ub, x_b, integer_val):
        self.c = c
        self.a_ub = a_ub
        self.b_ub = b_ub
        self.x_b = x_b
        self._integer_val = integer_val
        self.best_solution = float('inf')
        self.best_node = None
        self.nodes = []
        self.nodes_solution = []
 
    def solve_lp(self, cur_x_b):
        return linprog(self.c, A_ub=self.a_ub, b_ub=self.b_ub, bounds=cur_x_b)
 
    def check_fessible(self, res):
        if res['status'] == 0:
            return True
        elif res['status'] == 2:
            return False
        else:
            raise ("问题无界")
 
    def add_node(self, node):
        res = self.solve_lp(node._x_bounds)
        if self.check_fessible(res) and res['fun'] < self.best_solution:
            node.set_lp_res(res)
            self.nodes_solution.append(res['fun'])
            self.nodes.append(node)
            if node.check_integer_val_solved(len(self._integer_val)):
                self.best_solution = res['fun']
                self.best_node = node
                print("----------------当前解决方案-------------------")
                print("x: ", node._res['x'])
                print("z: ", node._res['fun'])
                print("---------------------------------------------------\n")
            print("==> 将节点添加到树列表: ", node._index)
            print("==> 当前节点: ", self.nodes_solution)
            print("")
            return True
        else:
            print("==> 节点不可行: ", node._index)
            print("==> 当前节点: ", self.nodes_solution)
            print("")
            return False
 
    def del_higher_val_node(self, z_s):
        del_list = []
        for i in range(len(self.nodes_solution)):
            if self.nodes_solution[i] >= z_s:
                del_list.append(i)
        s = ""
        for i in del_list:
            s += " " + str(self.nodes[i]._index)
        print("删除节点: ", s)
        self.nodes = list(np.delete(self.nodes, del_list))
        self.nodes_solution = list(np.delete(self.nodes_solution, del_list))
        print("当前节点: ", self.nodes_solution)
        print("")
 
    def del_item(self, index):
        print("删除节点: ", self.nodes[index]._index)
        self.nodes = list(np.delete(self.nodes, index))
        self.nodes_solution = list(np.delete(self.nodes_solution, index))
        print("当前节点: ", self.nodes_solution)
        print("")
 
    def check_bounds(self, temp_x_b, index, u_or_l):
        if u_or_l == 1:
            if self.x_b[index][0] is not None and temp_x_b[index][0] is None:
                return False
            elif self.x_b[index][0] is None and temp_x_b[index][0] is not None:
                return True
            elif self.x_b[index][0] is not None and temp_x_b[index][0] is not None:
                return False if(self.x_b[index][0] > temp_x_b[index][0]) else True
        elif u_or_l == 2:
            if self.x_b[index][1] is not None and temp_x_b[index][1] is None:
                return False
            elif self.x_b[index][1] is None and temp_x_b[index][1] is not None:
                return True
            elif self.x_b[index][1] is not None and temp_x_b[index][1] is not None:
                return False if(self.x_b[index][1] < temp_x_b[index][1]) else True
        else:
            print("界限误差")
            exit()
 
    def run(self):
        print("####################### 开始 B & B #####################\n")
        node_count = 0
        node = Node(copy.deepcopy(self.x_b), [], node_count)
        node_count += 1
        res = self.solve_lp(self.x_b)
 
        lower = floor(res['x'][self._integer_val[0]])
        upper = lower + 1
 
        lower_node = Node(copy.deepcopy(self.x_b), [], node_count, 1)
        lower_node.freeze_var(self._integer_val[0], lower)
        self.add_node(lower_node)
        node_count += 1
 
        upper_node = Node(copy.deepcopy(self.x_b), [], node_count, 2)
        upper_node.freeze_var(self._integer_val[0], upper)
        self.add_node(upper_node)
        node_count += 1
 
        while len(self.nodes) > 0:
            index = np.argmin(self.nodes_solution)
            x_b = self.nodes[index]._x_bounds
            freeze_list = self.nodes[index]._freeze_var_list
            res = self.nodes[index]._res
            freeze_var_index = len(freeze_list)
 
            lower = floor(res['x'][self._integer_val[freeze_var_index]])
            upper = lower + 1
            lower_node = Node(copy.deepcopy(x_b), copy.deepcopy(freeze_list), node_count, 1)
            lower_node.freeze_var(self._integer_val[freeze_var_index], lower)
            self.add_node(lower_node)
            node_count += 1
 
            upper_node = Node(copy.deepcopy(x_b), copy.deepcopy(freeze_list), node_count, 2)
            upper_node.freeze_var(self._integer_val[freeze_var_index], upper)
            self.add_node(upper_node)
            node_count += 1
 
            self.del_item(index)
            self.del_higher_val_node(self.best_solution)
            print("############################################################")
        print("")
        print("######################### 最佳解决方案 #######################")
        print(self.best_node._res)
 
 
if __name__ == "__main__":
    integer_val = [0,1]#整数变量索引列表
    c = [-40, -90]#目标函数系数
    A = [[9, 7], [7, 20]]#不等式约束的系数矩阵
    b = [56,70]#不等式约束的常数项
    x_bounds = [[0, None] for _ in range(len(c))]#变量的界限
    bb_algorithm = BbAlgorithm(c, A, b, x_bounds, integer_val)
    bb_algorithm.run()


####################### 开始 B & B #####################

创建节点: 0

创建节点: 1

x:  [4.0] 2.1
==> 将节点添加到树列表:  1
==> 当前节点:  [-349.0]

创建节点: 2

x:  [5.0] 1.5714285714285714
==> 将节点添加到树列表:  2
==> 当前节点:  [-349.0, -341.42857142857144]

创建节点: 3

x:  [4.0][2.0]
----------------当前解决方案-------------------
x:  [4. 2.]
z:  -340.0
---------------------------------------------------

==> 将节点添加到树列表:  3
==> 当前节点:  [-349.0, -341.42857142857144, -340.0]

创建节点: 4

==> 节点不可行:  4
==> 当前节点:  [-349.0, -341.42857142857144, -340.0]

删除节点:  1
当前节点:  [-341.42857142857144, -340.0]

删除节点:   3
当前节点:  [-341.42857142857144]

############################################################
创建节点: 5

==> 节点不可行:  5
==> 当前节点:  [-341.42857142857144]

创建节点: 6

==> 节点不可行:  6
==> 当前节点:  [-341.42857142857144]

删除节点:  2
当前节点:  []

删除节点:  
当前节点:  []

############################################################

######################### 最佳解决方案 #######################
        message: Optimization terminated successfully. (HiGHS Status 7: 

In [ ]:
import pulp

# 定义问题
problem = pulp.LpProblem("Mixed_Integer_Problem", pulp.LpMinimize)

# 定义连续变量
x = pulp.LpVariable('x', lowBound=0, cat='Continuous')

# 定义整数变量
y = pulp.LpVariable('y', lowBound=0, cat='Integer')

# 定义二元变量
z = pulp.LpVariable('z', cat='Binary')

# 添加目标函数
problem += x + 2*y + 3*z

# 添加约束条件
problem += x + y <= 10
problem += x - z >= 5

# 求解问题
problem.solve()


例题 1： 某厂生产甲乙两种饮料，每百箱甲饮料需用原料 6千克、工人 10名，获利 10万元；每百箱乙饮料需用原料 5千克、工人 20名，获利 9万元。 今工厂共有原料 60千克、工人 150名，又由于其他条件所限甲饮料产量不超过8百箱。  
问题 1：问如何安排生产计划，即两种饮料各生产多少使获利最大？  
问题 2：若投资0.8万元可增加原料1千克，是否应作这项投资？投资多少合理？  
问题 3：若不允许散箱（按整百箱生产），如何安排生产计划，即两种饮料各生产多少使获利最大？  
问题 4：若不允许散箱（按整百箱生产），若投资0.8万元可增加原料1千克，是否应作这项投资？投资多少合理？

In [3]:
import pulp      # 导入 pulp 库
# 问题 1：
"""
问题建模：
        决策变量：
            x1：甲饮料产量（单位：百箱）
            x2：乙饮料产量（单位：百箱）
        目标函数：
            max fx = 10*x1 + 9*x2
        约束条件：
            6*x1 + 5*x2 <= 60
            10*x1 + 20*x2 <= 150            
            x1, x2 >= 0，x1 <= 8
    此外，由 x1,x2>=0 和 10*x1+20*x2<=150 可知 0<=x2<=7.5
"""
ProbLP1 = pulp.LpProblem("ProbLP1", sense=pulp.LpMaximize)    # 定义问题 1，求最大值，sense定义目标函数最大值or最小值
x1 = pulp.LpVariable('x1', lowBound=0, upBound=8, cat='Continuous')  # 定义 x1
x2 = pulp.LpVariable('x2', lowBound=0, upBound=7.5, cat='Continuous')  # 定义 x2
ProbLP1 += (10*x1 + 9*x2)  # 设置目标函数 f(x)
ProbLP1 += (6*x1 + 5*x2 <= 60)  # 不等式约束
ProbLP1 += (10*x1 + 20*x2 <= 150)  # 不等式约束
ProbLP1.solve()
print(ProbLP1.name)  # 输出求解状态
print("Status youcans:", pulp.LpStatus[ProbLP1.status])  # 输出求解状态
for v in ProbLP1.variables():
    print(v.name, "=", v.varValue)  # 输出每个变量的最优值
print("F1(x) =", pulp.value(ProbLP1.objective))  # 输出最优解的目标函数值

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/09/wbb5c_x5097glbw2s6c8vcj40000gn/T/0c2c9ac6efd942d5ab004f7a3a958154-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/09/wbb5c_x5097glbw2s6c8vcj40000gn/T/0c2c9ac6efd942d5ab004f7a3a958154-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 14 RHS
At line 17 BOUNDS
At line 20 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
0  Obj -0 Dual inf 19 (2)
2  Obj 102.85714
Optimal - objective value 102.85714
Optimal objective 102.8571429 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

ProbLP1


In [5]:
# 问题 2：
"""
    问题建模：
        决策变量：
            x1：甲饮料产量（单位：百箱）
            x2：乙饮料产量（单位：百箱）
            x3：增加投资（单位：万元）
        目标函数：
            max fx = 10*x1 + 9*x2 - x3
        约束条件：
            6*x1 + 5*x2 <= 60 + x3/0.8
            10*x1 + 20*x2 <= 150
            x1, x2, x3 >= 0，x1 <= 8
    此外，由 x1,x2>=0 和 10*x1+20*x2<=150 可知 0<=x2<=7.5
"""
ProbLP2 = pulp.LpProblem("ProbLP2", sense=pulp.LpMaximize)    # 定义问题 2，求最大值
x1 = pulp.LpVariable('x1', lowBound=0, upBound=8, cat='Continuous')  # 定义 x1
x2 = pulp.LpVariable('x2', lowBound=0, upBound=7.5, cat='Continuous')  # 定义 x2
x3 = pulp.LpVariable('x3', lowBound=0, cat='Integer')  # 定义 x3
ProbLP2 += (10*x1 + 9*x2 - x3)  # 设置目标函数 f(x)
ProbLP2 += (6*x1 + 5*x2 - 1.25*x3 <= 60)  # 不等式约束
ProbLP2 += (10*x1 + 20*x2 <= 150)  # 不等式约束
ProbLP2.solve()
print(ProbLP2.name)  # 输出求解状态
print("Status  youcans:", pulp.LpStatus[ProbLP2.status])  # 输出求解状态
for v in ProbLP2.variables():
    print(v.name, "=", v.varValue)  # 输出每个变量的最优值
print("F2(x) =", pulp.value(ProbLP2.objective))  # 输出最优解的目标函数值

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/09/wbb5c_x5097glbw2s6c8vcj40000gn/T/2dcc8f340cbd4591b08c69b8db21ecbd-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/09/wbb5c_x5097glbw2s6c8vcj40000gn/T/2dcc8f340cbd4591b08c69b8db21ecbd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 18 RHS
At line 21 BOUNDS
At line 25 ENDATA
Problem MODEL has 2 rows, 3 columns and 5 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 107.1 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 2 rows, 3 columns (1 integer (0 of which binary)) and 5 elements
Cbc0012I Integer solution of -106.5 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 secon

In [6]:
# 问题 3：整数规划问题
"""
    问题建模：
        决策变量：
            x1：甲饮料产量，正整数（单位：百箱）
            x2：乙饮料产量，正整数（单位：百箱）
        目标函数：
            max fx = 10*x1 + 9*x2
        约束条件：
            6*x1 + 5*x2 <= 60
            10*x1 + 20*x2 <= 150
            x1, x2 >= 0，x1 <= 8，x1, x2 为整数
    此外，由 x1,x2>=0 和 10*x1+20*x2<=150 可知 0<=x2<=7.5
"""
import pulp
ProbLP3 = pulp.LpProblem("ProbLP3", sense=pulp.LpMaximize)  # 定义问题 3，求最大值
print(ProbLP3.name)  # 输出求解状态
x1 = pulp.LpVariable('x1', lowBound=0, upBound=8, cat='Integer')  # 定义 x1，变量类型：整数
x2 = pulp.LpVariable('x2', lowBound=0, upBound=7.5, cat='Integer')  # 定义 x2，变量类型：整数
ProbLP3 += (10 * x1 + 9 * x2)  # 设置目标函数 f(x)
ProbLP3 += (6 * x1 + 5 * x2 <= 60)  # 不等式约束
ProbLP3 += (10 * x1 + 20 * x2 <= 150)  # 不等式约束
ProbLP3.solve()
print("Shan Status:", pulp.LpStatus[ProbLP3.status])  # 输出求解状态
for v in ProbLP3.variables():
    print(v.name, "=", v.varValue)  # 输出每个变量的最优值
print("F3(x) =", pulp.value(ProbLP3.objective))  # 输出最优解的目标函数值

ProbLP3
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/09/wbb5c_x5097glbw2s6c8vcj40000gn/T/54421a4e2cf14d58a7b2c2a3c7db99cf-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/09/wbb5c_x5097glbw2s6c8vcj40000gn/T/54421a4e2cf14d58a7b2c2a3c7db99cf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 18 RHS
At line 21 BOUNDS
At line 24 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 102.857 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (2 integer (0 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of -96 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0012I Integer

In [8]:
# 问题 4：
"""
    问题建模：
        决策变量：
            x1：甲饮料产量，正整数（单位：百箱）
            x2：乙饮料产量，正整数（单位：百箱）
            x3：增加投资（单位：万元）
        目标函数：
            max fx = 10*x1 + 9*x2 - x3
        约束条件：
            6*x1 + 5*x2 <= 60 + x3/0.8
            10*x1 + 20*x2 <= 150
            x1, x2, x3 >= 0，x1 <= 8，x1, x2 为整数
    此外，由 x1,x2>=0 和 10*x1+20*x2<=150 可知 0<=x2<=7.5
"""
ProbLP4 = pulp.LpProblem("ProbLP4", sense=pulp.LpMaximize)  # 定义问题 4，求最大值
print(ProbLP4.name)  # 输出求解状态
x1 = pulp.LpVariable('x1', lowBound=0, upBound=8, cat='Integer')  # 定义 x1，变量类型：整数
x2 = pulp.LpVariable('x2', lowBound=0, upBound=7, cat='Integer')  # 定义 x2，变量类型：整数
x3 = pulp.LpVariable('x3', lowBound=0, cat='Integer')  # 定义 x3
ProbLP4 += (10*x1 + 9*x2 - x3)  # 设置目标函数 f(x)
ProbLP4 += (6*x1 + 5*x2 - 1.25*x3 <= 60)  # 不等式约束
ProbLP4 += (10*x1 + 20*x2 <= 150)  # 不等式约束
ProbLP4.solve()
print("Shan Status:", pulp.LpStatus[ProbLP4.status])  # 输出求解状态
for v in ProbLP4.variables():
    print(v.name, "=", v.varValue)  # 输出每个变量的最优值
print("F4(x) =", pulp.value(ProbLP4.objective))  # 输出最优解的目标函数值

ProbLP4
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/09/wbb5c_x5097glbw2s6c8vcj40000gn/T/c098ec7332f544b3b4e346851d8fe7ef-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/09/wbb5c_x5097glbw2s6c8vcj40000gn/T/c098ec7332f544b3b4e346851d8fe7ef-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 22 RHS
At line 25 BOUNDS
At line 29 ENDATA
Problem MODEL has 2 rows, 3 columns and 5 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 107.1 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 2 rows, 3 columns (3 integer (0 of which binary)) and 5 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of -102 found by Dive